In [1]:
# Imports
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as pl

from keras import backend
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19

from scipy.optimize import fmin_l_bfgs_b
        
import tensorflow as tf
assert tf.__version__[0] == '1', 'tensorflow version 1 is required'

from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import datetime

In [0]:
#!ls -al drive/'My Drive'/'AIschool Team4'/이규원/input_image
#!ls -al drive/'My Drive'/'AIschool Team4'/이규원/style_image
#!cp -r drive/'My Drive'/'AIschool Team4'/이규원/기본/* drive/'My Drive'/'AIschool Team4'/이규원/


In [0]:
# Hyperparams
ITERATIONS = 20
CHANNELS = 3
IMAGE_SIZE = 500
IMAGE_WIDTH = IMAGE_SIZE
IMAGE_HEIGHT = IMAGE_SIZE
IMAGENET_MEAN_RGB_VALUES = [123.68, 116.779, 103.939]
CONTENT_WEIGHT = 0.02 # default = 0.02
STYLE_WEIGHT = 4.5    # default = 4.5
TOTAL_VARIATION_WEIGHT = 0.995 # default = 0.995
TOTAL_VARIATION_LOSS_FACTOR = 1.25 # default = 1.25

In [0]:
'''define functions'''
### I don't understand why backend is used.

def load_img(path, IMAGE_WIDTH, IMAGE_HEIGHT):
    image = Image.open(path)
    image = image.resize((IMAGE_WIDTH, IMAGE_HEIGHT))
    image.save(path)
    return image

def data_norm_reshap(image):
    image_array = np.asarray(image, dtype="float32") # shape == (500, 500, 3)
    image_array = np.expand_dims(image_array, axis=0) # shape == (1, 500, 500, 3)

    image_array[:, :, :, 0] -= IMAGENET_MEAN_RGB_VALUES[2] # normalization
    image_array[:, :, :, 1] -= IMAGENET_MEAN_RGB_VALUES[1]
    image_array[:, :, :, 2] -= IMAGENET_MEAN_RGB_VALUES[0]
    image_array = image_array[:, :, :, ::-1] # reshaping
  
    return image_array

def mk_model(input_image_array, style_image_array, IMAGE_HEIGHT, IMAGE_SIZE):
    input_image = backend.variable(input_image_array) # content image variable | shape == (1, 500, 500, 3)
    style_image = backend.variable(style_image_array) # style image variable | shape == (1, 500, 500, 3)
    combination_image = backend.placeholder((1, IMAGE_HEIGHT, IMAGE_SIZE, 3)) # placeholder..? | shape == (1, 500, 500, 3)

    input_tensor = backend.concatenate([input_image,style_image,combination_image], axis=0)
#                      concatenated in order of content(0), style(1), noise(2)
#   model = VGG16(input_tensor=input_tensor, include_top=False) # WARNING : VGG16
    model = VGG19(input_tensor=input_tensor, include_top=False)
    return input_image, style_image, combination_image, model

def content_loss(content, combination):
    return backend.sum(backend.square(combination - content))

def gram_matrix(x):
    features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))
    gram = backend.dot(features, backend.transpose(features))
    return gram

def compute_style_loss(style, combination):
    style = gram_matrix(style)
    combination = gram_matrix(combination)
    size = IMAGE_HEIGHT * IMAGE_WIDTH
    return backend.sum(backend.square(style - combination)) / (4. * (CHANNELS ** 2) * (size ** 2))

def total_variation_loss(x):
    '''
    x : combination_image (== noise image)
    combination_image = backend.placeholder((1, IMAGE_HEIGHT, IMAGE_SIZE, 3)) # placeholder..? | shape == (1, 500, 500, 3)
    what is placeholder?
    '''
    a = backend.square(x[:, :IMAGE_HEIGHT-1, :IMAGE_WIDTH-1, :] - x[:, 1:, :IMAGE_WIDTH-1, :])
    b = backend.square(x[:, :IMAGE_HEIGHT-1, :IMAGE_WIDTH-1, :] - x[:, :IMAGE_HEIGHT-1, 1:, :])
    return backend.sum(backend.pow(a + b, TOTAL_VARIATION_LOSS_FACTOR))

def evaluate_loss_and_gradients(x):
    x = x.reshape((1, IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
    outs = backend.function([combination_image], outputs)([x]) ### backend.function..?
    loss = outs[0]
    gradients = outs[1].flatten().astype("float64")
    return loss, gradients

class Evaluator:

    def loss(self, x):
        loss, gradients = evaluate_loss_and_gradients(x)
        self._gradients = gradients
        return loss

    def gradients(self, x):
        return self._gradients

def visualx(x):
    '''x : normalized numpy array'''
    x = x.copy()
    x = x.reshape((IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
    x = x[:, :, ::-1]
    x[:, :, 0] += IMAGENET_MEAN_RGB_VALUES[2]
    x[:, :, 1] += IMAGENET_MEAN_RGB_VALUES[1]
    x[:, :, 2] += IMAGENET_MEAN_RGB_VALUES[0]
    x = np.clip(x, 0, 255).astype("uint8")
    output_image = Image.fromarray(x)
    return output_image
  
def noise(IMAGE_HEIGHT, IMAGE_WIDTH, noise_image_path):
    x = np.random.uniform(0, 255, (1, IMAGE_HEIGHT, IMAGE_WIDTH, 3)) - 128. # noise image??
    noise_image = visualx(x)
    noise_image.save(noise_image_path)
    return x, noise_image

In [0]:
# Paths
#input_image_path = "drive/My Drive/Alschool Team4/이규원/input_image/"
#style_image_path = "drive/My Drive/Alschool Team4/이규원/style_image/"

input_image_name = ['statue-of-liberty']                    
style_image_name = ['bridg', 'evening', 'flowers', 'planet', 'post-impressionist']#png = '.png'


for input in input_image_name:
  for style in style_image_name:
    
    input_image_path = 'drive/My Drive/AIschool Team4/이규원/input_image/'+input+'.png'
    style_image_path = 'drive/My Drive/AIschool Team4/이규원/style_image/'+style+'.png'
    output_image_path = 'drive/My Drive/AIschool Team4/이규원/output_image/'+style+'ed_'+input+'.png'
    combined_image_path = 'drive/My Drive/AIschool Team4/이규원/'+'combined_'+style+'ed_'+input+'.png'

    folder_name = style+'ed_'+input
    !mkdir 'drive/My Drive/AIschool Team4/이규원/canvas_original_100/'$folder_name
    
    
    canvas_path = 'drive/My Drive/AIschool Team4/이규원/canvas_original_100/'+folder_name+'/canvas_iter{}.png'
    noise_image_path = 'drive/My Drive/AIschool Team4/이규원/canvas_original_100/'+folder_name+'/noise.png'
  
    C_path = input_image_path
    S_path = style_image_path
  
    print('***************', style+'ed_'+input, '***************', datetime.datetime.now())
  
    input_image = load_img(C_path, IMAGE_HEIGHT, IMAGE_WIDTH)
    style_image = load_img(S_path, IMAGE_HEIGHT, IMAGE_WIDTH)
    
    input_image_array = data_norm_reshap(input_image)
    style_image_array = data_norm_reshap(style_image)
    input_image, sytle_image, combination_image, model = mk_model(input_image_array, style_image_array, IMAGE_HEIGHT, IMAGE_SIZE)
  
    layers = dict([(layer.name, layer.output) for layer in model.layers])
    content_layer = "block2_conv2" # which layer to be used for content_loss
    layer_features = layers[content_layer] # output of content_layer
    content_image_features = layer_features[0, :, :, :] # feature map of content image in block2_conv2 (== content_layer)
    combination_features = layer_features[2, :, :, :] # feature map of noise iamge in block2_conv2 (== content_layer)
    loss = backend.variable(0.)
    loss += CONTENT_WEIGHT * content_loss(content_image_features, combination_features)
  
    style_layers = ["block1_conv2", "block2_conv2", "block3_conv3", "block4_conv3", "block5_conv3"]
    for layer_name in style_layers:
        layer_features = layers[layer_name] # output of style_layers
        style_features = layer_features[1, :, :, :] # feature map of style image
        combination_features = layer_features[2, :, :, :] # feature map of noise image
        style_loss = compute_style_loss(style_features, combination_features)
        loss += (STYLE_WEIGHT / len(style_layers)) * style_loss
    
    loss += TOTAL_VARIATION_WEIGHT * total_variation_loss(combination_image)

    outputs = [loss] ### what's this?
    outputs += backend.gradients(loss, combination_image)
  
    evaluator = Evaluator()
  
    x, noise_image = noise(IMAGE_HEIGHT, IMAGE_WIDTH, noise_image_path)
    
    for i in range(ITERATIONS):
        x, loss, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.gradients, maxfun=20)
        canvas = visualx(x)
        canvas.save(canvas_path.format(i))
        print("Iteration %d completed with loss %d" % (i, loss))
    
    x = x.reshape((IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
    x = x[:, :, ::-1]
    x[:, :, 0] += IMAGENET_MEAN_RGB_VALUES[2]
    x[:, :, 1] += IMAGENET_MEAN_RGB_VALUES[1]
    x[:, :, 2] += IMAGENET_MEAN_RGB_VALUES[0]
    x = np.clip(x, 0, 255).astype("uint8")
    output_image = Image.fromarray(x)
    output_image.save(output_image_path)

    combined = Image.new("RGB", (IMAGE_WIDTH*3, IMAGE_HEIGHT))
    x_offset = 0
    for image in map(Image.open, [input_image_path, style_image_path, output_image_path]):
        combined.paste(image, (x_offset, 0))
        x_offset += IMAGE_WIDTH
    combined.save(combined_image_path)
    print('***************', 'completed','***************')

mkdir: cannot create directory ‘drive/My Drive/AIschool Team4/이규원/canvas_original_100/bridged_statue-of-liberty’: File exists
*************** bridged_statue-of-liberty *************** 2019-08-19 13:41:14.941269


W0819 13:41:16.814781 140568526956416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0819 13:41:16.821798 140568526956416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0819 13:41:16.825032 140568526956416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0819 13:41:16.877082 140568526956416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



80142336/80134624 [==============================] - 7s 0us/step


W0819 13:41:25.752739 140568526956416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0819 13:41:25.754156 140568526956416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0819 13:41:28.926808 140568526956416 variables.py:2429] Variable += will be deprecated. Use variable.assign_add if you want assignment to the variable value or 'x = x + y' if you want a new python Tensor object.
W0819 13:41:29.193573 140568526956416 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updatin

Iteration 0 completed with loss 41186496512
Iteration 1 completed with loss 14795197440
Iteration 2 completed with loss 11052460032
Iteration 3 completed with loss 9755061248
Iteration 4 completed with loss 9094660096
Iteration 5 completed with loss 8496013312
Iteration 6 completed with loss 7974195200
Iteration 7 completed with loss 7404662784
Iteration 8 completed with loss 6812642816
Iteration 9 completed with loss 6177075200
Iteration 10 completed with loss 5718360576
Iteration 11 completed with loss 5197266432
Iteration 12 completed with loss 4801646592
Iteration 13 completed with loss 4458897920
Iteration 14 completed with loss 4204295680
Iteration 15 completed with loss 4012777984
Iteration 16 completed with loss 3847870208
Iteration 17 completed with loss 3700473088
Iteration 18 completed with loss 3586078720
Iteration 19 completed with loss 3491884288
*************** completed ***************
*************** eveninged_statue-of-liberty *************** 2019-08-19 13:52:03.50301